Goal of analysis script is to pull in Med-PC files from Pavlovian conditioning experiment and pull out event times and stamps into a df

Import numpy

In [17]:
import numpy as np
import pandas as pd
import re

Open dialog box and prompt user to select directory containing the data files

In [18]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav7


Use os module to create list of data folders

In [19]:
import os
session_list = os.listdir(root.directory)
session_list

['180628_266_Pav7',
 '180628_268_Pav7',
 '180628_269_Pav7',
 '180628_270_Pav7',
 '180628_271_Pav7',
 '180628_273_Pav7',
 '180628_274_Pav7',
 '180628_275_Pav7',
 '180628_277_Pav7',
 '180628_278_Pav7',
 '180628_280_Pav7',
 '180628_283_Pav7',
 '180628_287_Pav7',
 '180628_288_Pav7',
 '180628_289_Pav7',
 '180628_290_Pav7',
 '180628_292_Pav7',
 '180629_Pav7_267',
 '180629_Pav7_282',
 '180629_Pav7_291']

In [21]:
file_paths = []
for file in session_list:
    file_path = root.directory + '/' + file
    file_paths.append(file_path)

file_paths[1]

'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav7/180628_268_Pav7'

In [46]:
data = pd.read_table(file_paths[0])
data_table = pd.DataFrame(data = data)
data_array = data_table.values

data_array

array([['Start Date: 06/28/18'],
       ['End Date: 06/28/18'],
       ['Subject: 266'],
       ['Experiment: 0'],
       ['Group: 0'],
       ['Box: 1'],
       ['Start Time: 12:48:33'],
       ['End Time: 13:45:41'],
       ['MSN: AS_PavlovianTask'],
       ['F:       0.000'],
       ['G:     537.000'],
       ['H:    1732.890'],
       ['J:       0.000'],
       ['K:       0.000'],
       ['M:       0.000'],
       ['O:       0.000'],
       ['Q:       0.000'],
       ['R:       0.000'],
       ['S:       0.000'],
       ['U:       0.000'],
       ['W:       0.000'],
       ['X:       0.000'],
       ['Y:       0.000'],
       ['Z:       0.000'],
       ['A:'],
       ['     0:        0.000        0.000        0.000        0.000        1.000'],
       ['     5:        0.500        1.000       60.000       10.000        5.000'],
       ['    10:        5.000        0.000       25.000        0.000        0.000'],
       ['    15:        3.000        0.000       10.000        2.000    

In [47]:
def get_eandt(array):
    event_letter = "E"
    time_letter = "T"
    re_l = r"..[A-Z]...$"
    re_n = r'.. '

    i = 0
    j = 0
    k = 0
    event_data = []
    time_data = []
    for row in array:
        if re.search(re_l, str(row)):
            marker = str(row) 
            j = i
            k = i
            if re.match(marker, event_letter):
                j = j + 1
                while re.search(re_n, str(array[j])):
                    event_data.append((str(array[j])))
                    j = j + 1
            if re.match(marker, time_letter):
                k = k + 1
                while re.search(re_n, str(array[k])):
                    time_data.append((str(array[k])))
                    k = k + 1
            else:
                i = i + 1
        else:
            i = i + 1
            
    re_n = r'^\d(\d|\.)(\d|\.)(\d|\.)'
    event_data_list = []
    for row in event_data:
        data_set_split = row.split(" ")
        for line in data_set_split:
            if re.search(re_n, line):
                event_data_list.append(float(line[0:(len(line)-2)]))
                
    re_n = r'^\d(\d|\.)(\d|\.)(\d|\.)'
    time_data_list = []
    for row in time_data:
        data_set_split = re.split(" |'", row)
        for line in data_set_split:
            if re.search(re_n, line):
                time_data_list.append(float(line[0:(len(line)-1)]))

    final_data = np.column_stack((event_data_list, time_data_list))
    
    return final_data

In [48]:
es_et = pd.DataFrame(data = get_eandt(data_array))
es_et.columns = ['event_stamp', 'time_stamp']
es_et.head()

,event_stamp,time_stamp
0,15.0,0.02
1,16.0,40.02
2,22.0,40.03
3,18.0,40.03
4,7.0,40.03


\  Event stamps ( array E )
\   1   = Rt lever press/NP
\   2   = Lt lever press/NP
\   5   = Pellet reinforcement delivery
\   6   = Food tray head entry
\   7   = Right cue light on
\   8   = Left cue light on
\   9   = Right cue light off
\   10  = Left cue ight off
\   11  = Magazine tray light on
\   12  = Magazine tray light off
\   13  = Liquid delivery turns on
\   14  = Liquid delivery turns off
\   15  = ITI starts
\   16  = ITI ends
\   18  = Right lever extends
\   19  = Left lever extends
\   20  = Right lever retracts
\   21  = Left lever retracts
\   22  = CS on
\   23  = CS off
\   100 = Session termination
\   999 = Place holder - no event

Make list of CS and ITI start and end times

In [49]:
cs_start = es_et[es_et.event_stamp == 22].reset_index()
cs_end = es_et[es_et.event_stamp == 23].reset_index()
ITI_start = es_et[es_et.event_stamp == 15].reset_index()
ITI_end = es_et[es_et.event_stamp == 16].reset_index()

Make list into dataframe CS and ITI start and end times

In [50]:
cs_iti = pd.DataFrame(data = [cs_start.time_stamp, cs_end.time_stamp, ITI_start.time_stamp[:-1], ITI_end.time_stamp])
cs_iti = cs_iti.T
cs_iti.columns = ['CS_start', 'CS_end', 'ITI_start', 'ITI_end']
cs_iti

,CS_start,CS_end,ITI_start,ITI_end
0,40.03,50.02,0.02,40.02
1,100.14,110.13,50.13,100.13
2,200.26,210.25,110.25,200.25
3,290.37,300.36,210.36,290.36
4,370.48,380.47,300.47,370.47
5,440.59,450.58,380.58,440.58
6,480.70,490.69,450.69,480.69
7,560.83,570.82,490.82,560.82
8,660.95,670.94,570.94,660.94
9,701.07,711.06,671.06,701.06


we routinely measure 
(a) the number of lever deflections and food cup head entries during the CS period, 
count number of lever pressess, count number of food cup head entries

(b) the probability of contacting the lever or
entering the food cup during the CS period on each trial (defined
as the number of trials with a lever press or food cup entry, divided
by the total number of trials). count numnber of trials a lp or he occured then divide by number of trials

(c) the ave latency to the first lever
deflection or food cup entry. find latency for first lp or he for each trial then average over number of trials with lp of he

Using these measures we devised a
‘‘PCA Score’’, as follows. From the CS lever deflections and CS
food cup entries, we calculated three measures of approach: 

1) Response Bias (ratio of lever presses and food cup entries in relation
to total number of responses), 

2) Probability Difference (the difference
between the probability of pressing the lever and the probability of
entering the food cup), and 

3) Latency Score (difference between the
latencies to approach the lever and the food cup). 

Averaging these
three measures of approach produces a PCA Score for each
individual, on each day of training. 

Scores derived this way range
from 21 to +1, whereby scores of 21 and +1 indicate strong
biases toward goal-tracking and sign-tracking, respectively, and a
score of zero indicates that the two responses are equally
distributed. Table 1 describes these calculations in detail. 

We also
measured the number of food cup head entries during the intertrial
interval (ITI), but these are not included as part of the PCA
Score.

Use dataframe of start and end times to select events during each CS and compute PCA parameters

In [51]:
PCA_values = pd.DataFrame()

lever = 1
lever_press = 0
head_entries = 0
lever_prob = 0
head_prob = 0
ave_lat_lever = 0
ave_lat_head = 0

count_lever = 0
count_head = 0
lat_lever_sum = 0
lat_head_sum = 0

response_bias = 0
prob_diff = 0
lat_score = 0 
PCA_score = 0

i = 0

while i < 25:
    data = es_et[(es_et.time_stamp >= cs_iti.CS_start[i]) & (es_et.time_stamp <= cs_iti.CS_end[i])]
    
    if len(data[data.event_stamp == 1]) > 0:
        lever == 1
    if len(data[data.event_stamp == 2]) > 0:
        lever == 2
        
    lever_press = lever_press + len(data[data.event_stamp == lever])
    
    head_entries = head_entries + len(data[data.event_stamp == 6])

    if len(data[data.event_stamp == lever]) > 0:
        print(i)
        count_lever = count_lever + 1
        lat_lever_sum = lat_lever_sum + data[data.event_stamp == lever].time_stamp.iloc[0]
        
    if len(data[data.event_stamp == 6]) > 0:
        count_head = count_head + 1
        lat_head_sum = lat_head_sum + data[data.event_stamp == 6].time_stamp.iloc[0]
        
    i = i + 1
    
lever_prob = count_lever / 25
head_prob = count_head / 25

if lat_lever_sum > 0:
    ave_lat_lever = lat_lever_sum / count_lever * .00225
if lat_head_sum > 0:
    ave_lat_head = lat_head_sum / count_head * .00225

response_bias = (lever_press - head_entries) / (lever_press + head_entries)
prob_diff = lever_prob - head_prob
lat_score = (ave_lat_lever - ave_lat_head) / 10

PCA_score = (response_bias + prob_diff + lat_score) / 3

PCA_values['lever_press'] = [lever_press]
PCA_values['head_entries'] = head_entries
PCA_values['lever_prob'] = lever_prob
PCA_values['head_prob'] = head_prob
PCA_values['ave_lat_lever'] = ave_lat_lever
PCA_values['ave_lat_head'] = ave_lat_head
PCA_values['response_bias'] = response_bias
PCA_values['prob_diff'] = prob_diff
PCA_values['PCA_score'] = PCA_score

PCA_values

,lever_press,head_entries,lever_prob,head_prob,ave_lat_lever,ave_lat_head,response_bias,prob_diff,PCA_score
0,0,61,0.0,0.44,0,2.385925,-1.0,-0.44,-0.559531
